In [1]:
# libraries

import pandas as pd
import glob
import numpy as np
from pandasql import sqldf 
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
mysql = lambda q: sqldf(q, globals())
from sklearn.linear_model import (LinearRegression, 
                                  HuberRegressor, 
                                  RANSACRegressor, 
                                  TheilSenRegressor)
import statsmodels.api as sm

C:\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\miniconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\miniconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
absolute_path = 'C:/Users/user/OneDrive/Υπολογιστής/Αγροτικα/MeteoData'
os.chdir(absolute_path)

In [3]:
print('New working directory is: ', os.getcwd())

New working directory is:  C:\Users\user\OneDrive\Υπολογιστής\Αγροτικα\MeteoData


In [4]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!head -12 2011-05.txt

'head' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
!tail -5 2011-05.txt

'tail' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
%pwd

'C:\\Users\\user\\OneDrive\\Υπολογιστής\\Αγροτικα\\MeteoData'

In [8]:
# specify entire folder
folder_path = 'C:/Users/user/OneDrive/Υπολογιστής/Αγροτικα/MeteoData'

# create list of files
file_list = glob.glob(folder_path + "/*.txt")

# create pandas dataframe
main_dataframe = pd.read_fwf(file_list[0], header=None, widths=[3,7,6,8,5,8,7,6,5,7,7,8,6], skiprows=10)
main_dataframe.drop(main_dataframe.tail(2).index, inplace = True)

# populate pandas dataframe
for i in range(1, len(file_list)):
    data = pd.read_fwf(file_list[i], header=None, widths=[3,7,6,8,5,8,7,6,5,7,7,8,6], skiprows=10)
    df = pd.DataFrame(data)
    df.drop(df.tail(2).index, inplace = True)
    main_dataframe = pd.concat([main_dataframe,df], axis=0)

In [9]:
main_dataframe.shape

(4322, 13)

In [10]:
main_dataframe.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12
23,24,13.1,17.9,16:20,9.8,00:00,90,63,0.0,0.2,16.1,15:10,NE
24,25,14.4,22.9,15:40,7.8,06:30,88,37,0.0,0.4,22.5,16:50,ESE
25,26,17.4,22.0,15:40,13.6,01:40,80,40,0.0,3.7,45.1,16:10,SSE
26,27,17.5,22.0,15:10,12.8,07:40,84,52,0.0,1.5,24.1,02:50,ESE
27,28,14.0,16.1,16:20,11.7,00:00,88,79,0.0,0.3,14.5,12:10,W


In [13]:
# create date range
main_dataframe['date'] = pd.bdate_range(start='2011/05/01', end='2023/02/28', freq='D')

In [14]:
main_dataframe.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 'date'], dtype='object')

In [15]:
# remove unesecery columns
main_dataframe = main_dataframe.drop([0,6,7,9,10,11,12], axis=1)

In [16]:
main_dataframe.tail()

,1,2,3,4,5,8,date
23,13.1,17.9,16:20,9.8,00:00,0.0,2023-02-24
24,14.4,22.9,15:40,7.8,06:30,0.0,2023-02-25
25,17.4,22.0,15:40,13.6,01:40,0.0,2023-02-26
26,17.5,22.0,15:10,12.8,07:40,0.0,2023-02-27
27,14.0,16.1,16:20,11.7,00:00,0.0,2023-02-28


In [17]:
#rename columns
main_dataframe.rename(columns = {
                                    1:'mean_temp',
                                    2:'high_temp',
                                    3:'high_temp_time',
                                    4:'low_temp',
                                    5:'low_temp_time',
                                    8:'rain(mm)'
                                }, 
                                inplace = True
                     )

In [18]:
main_dataframe.columns

Index(['mean_temp', 'high_temp', 'high_temp_time', 'low_temp', 'low_temp_time',
       'rain(mm)', 'date'],
      dtype='object')

In [19]:
main_dataframe.tail(5)

,mean_temp,high_temp,high_temp_time,low_temp,low_temp_time,rain(mm),date
23,13.1,17.9,16:20,9.8,00:00,0.0,2023-02-24
24,14.4,22.9,15:40,7.8,06:30,0.0,2023-02-25
25,17.4,22.0,15:40,13.6,01:40,0.0,2023-02-26
26,17.5,22.0,15:10,12.8,07:40,0.0,2023-02-27
27,14.0,16.1,16:20,11.7,00:00,0.0,2023-02-28


In [20]:
main_dataframe.dtypes

mean_temp                 object
high_temp                 object
high_temp_time            object
low_temp                  object
low_temp_time             object
rain(mm)                  object
date              datetime64[ns]
dtype: object

In [21]:
# convert format 

main_dataframe['mean_temp'] = main_dataframe['mean_temp'].astype(float, errors = 'raise')
main_dataframe['high_temp'] = main_dataframe['high_temp'].astype(float, errors = 'raise')
main_dataframe['low_temp'] = main_dataframe['low_temp'].astype(float, errors = 'raise')
main_dataframe['rain(mm)'] = main_dataframe['rain(mm)'].astype(float, errors = 'raise')

In [22]:
# create new coluns
#df['month'] = df['purchase_date'].to_numpy().astype('datetime64[M]')

#main_dataframe['year_month'] = main_dataframe['date'].dt.strftime('%Y-%m')
main_dataframe['year_month'] = main_dataframe['date'].to_numpy().astype('datetime64[M]')
main_dataframe['month'] = main_dataframe['date'].dt.strftime('%m')
main_dataframe['year'] = main_dataframe['date'].dt.strftime('%Y')
main_dataframe['day'] = main_dataframe['date'].dt.strftime('%d')

In [23]:
main_dataframe.tail()

,mean_temp,high_temp,high_temp_time,low_temp,low_temp_time,rain(mm),date,year_month,month,year,day
23,13.1,17.9,16:20,9.8,00:00,0.0,2023-02-24,2023-02-01,02,2023,24
24,14.4,22.9,15:40,7.8,06:30,0.0,2023-02-25,2023-02-01,02,2023,25
25,17.4,22.0,15:40,13.6,01:40,0.0,2023-02-26,2023-02-01,02,2023,26
26,17.5,22.0,15:10,12.8,07:40,0.0,2023-02-27,2023-02-01,02,2023,27
27,14.0,16.1,16:20,11.7,00:00,0.0,2023-02-28,2023-02-01,02,2023,28


In [24]:
main_dataframe.dtypes

mean_temp                float64
high_temp                float64
high_temp_time            object
low_temp                 float64
low_temp_time             object
rain(mm)                 float64
date              datetime64[ns]
year_month         datetime64[s]
month                     object
year                      object
day                       object
dtype: object

----------